Library Imports: The list of open-source libraries which will be used in this project.

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math

Importing the List of Stocks (Static version of S&P 500)

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

Acquiring API (Sandbox mode of IEX cloud API)

In [ ]:
from secrets import IEX_CLOUD_API_TOKEN

Making an API call (Needs Market cap and price of the stock)

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

Parsing the API call

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap']

Adding our Stocks to the Data Frame

In [ ]:
my_columns = ['Ticker','Stock Price','Market Cap','Number of shares to buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

In [ ]:
final_dataframe.append(
    pd.Series(
   [ symbol,
    price,
    market_cap,
    'N/A'],
        index = my_columns
    ),ignore_index=True
)

Looping through the Tickers in the Stocks list

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],index = my_columns),
        ignore_index = True
    )

Using Batchwise API calls

In [61]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [62]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data =requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
    pd.Series(
    [
        symbol,
        data[symbol]['quote']['latestPrice'],
        data[symbol]['quote']['marketCap'],
        'N/A'
    ],index = my_columns),
        ignore_index = True
    )
final_dataframe
        

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,140.76,43067173276,N/A
1,AAL,14.00,9204566892,N/A
2,AAP,191.76,11660679197,N/A
3,AAPL,161.83,2542733637724,N/A
4,ABBV,154.79,266673500012,N/A
...,...,...,...,...
496,YUM,118.70,33675976104,N/A
497,ZBH,113.86,24851332230,N/A
498,ZBRA,301.92,15394462250,N/A
499,ZION,54.10,7836209519,N/A


Calculating the number of shares to buy

In [67]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [68]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of shares to buy'] = math.floor(position_size / final_dataframe['Stock Price'][i])
final_dataframe

,Ticker,Stock Price,Market Cap,Number of shares to buy
0,A,140.76,43067173276,141
1,AAL,14.00,9204566892,1425
2,AAP,191.76,11660679197,104
3,AAPL,161.83,2542733637724,123
4,ABBV,154.79,266673500012,128
...,...,...,...,...
496,YUM,118.70,33675976104,168
497,ZBH,113.86,24851332230,175
498,ZBRA,301.92,15394462250,66
499,ZION,54.10,7836209519,368
